# 0. 로그포맷 생성

In [8]:
from datetime import datetime
log_dict = {'실행 명령어': '판례요약-테스트.ipynb 처음부터 순차적으로 실행','TimeLog': {'시작': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}}
log_dict['최종 결과 값'] = {}
log_dict['계산할 때 사용된 값'] = {}
log_dict['파라미터'] = {
    'model_path': 'result/2024-02-21 10:36/',
    'checkpoint': 'checkpoint-26606',
    'num_beams':4,  
    'max_length':1024,  
    'eos_token_id':1, 
    'length_penalty':0.5
}
log_dict['문제당 개별 결과 값'] = []

print('TimeLog:', '시작', log_dict['TimeLog']['시작'])

TimeLog: 시작 2024-02-21 14:50:15


# 1. GPU 확인
- 현재 버전에서는 tesla T4 x 2 개의 GPU 로 학습하도록 설정하였습니다.
- tesla T4 x 1 로 학습 가능합니다. 

In [9]:
import os
import logging

log_dict['TimeLog']['1. GPU 확인'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print('TimeLog:', log_dict['TimeLog']['1. GPU 확인'])
print()

os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["TOKENIZERS_PARALLELISM"]="false"
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS']='true'

logging.disable(logging.INFO) 
logging.disable(logging.WARNING)

model_name = 'gogamza/kobart-base-v2'

!nvidia-smi

TimeLog: 2024-02-21 14:50:26

Wed Feb 21 14:50:26 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   36C    P0    26W /  70W |   1847MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:5E:00.0 Off |  

# 2. 전처리 함수 생성

In [10]:
import os
import re
import json
import tqdm
import glob
from bs4 import BeautifulSoup
from transformers import PreTrainedTokenizerFast

log_dict['TimeLog']['2. 전처리 함수 생성'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print('TimeLog:', log_dict['TimeLog']['2. 전처리 함수 생성'])
print()

def load_json_from_dir(dir_path):
    json_files = glob.glob(os.path.join(dir_path, '**/*.json'), recursive =True)
    json_data = []
    for file in json_files:
        with open(file, 'r', encoding='utf-8-sig') as f:
            temp_json = json.load(f)
            doc_id = str(temp_json['info']['id'])
            json_data.append(
                {
                    'id': doc_id,
                    'summ_contxt': temp_json['Summary'][0]['summ_contxt'],
                    'summ_pass': temp_json['Summary'][0]['summ_pass'],
                }
            )
    return json_data

def preprocessing(list_of_dict_data):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

    data_trimmed = []
    for data in list_of_dict_data:
        data['summ_contxt'] = data['summ_contxt'].replace('_x000D_', '').replace('\r', '').replace('\n', ' ').strip()
        data['summ_pass'] = data['summ_pass'].replace('_x000D_', '').replace('\r', '').replace('\n', ' ').strip()
        data_trimmed.append(data)
    return data_trimmed

TimeLog: 2024-02-21 14:50:28



# 3. 평가데이터 로드 및 전처리

In [11]:
log_dict['TimeLog']['3. 평가데이터 로드 및 전처리'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print('TimeLog:', log_dict['TimeLog']['3. 평가데이터 로드 및 전처리'])
print()

raw_test_json = load_json_from_dir('data/3.Test/')
test_dataset = preprocessing(raw_test_json)
print('평가데이터 수:', len(test_dataset))
print('평가데이터 샘플:') 
print(json.dumps(test_dataset[0], ensure_ascii=False, indent=2))

TimeLog: 2024-02-21 14:50:30

평가데이터 수: 6651
평가데이터 샘플:
{
  "id": "41018258",
  "summ_contxt": "함정수사라 함은 본래 범의를 가지지 아니한 자에 대하여 수사기관이 사술이나 계략 등을 써서 범의를 유발케 하여 범죄인을 검거하는 수사방법을 말하는 것이므로, 범의를 가진 자에 대하여 범행의 기회를 주거나 범행을 용이하게 한 것에 불과한 경우에는 함정수사라고 말할 수 없다(당원 1983. 4. 12. 선고 82도2433 판결 참조).",
  "summ_pass": "범의를 가진 자에 대하여 범행의 기회를 주거나 범행을 용이하게 한 것에 불과한 경우에는 함정수사라고 말할 수 없다."
}


# 4. 요약출력 함수 생성
- 선택된 모델을 통해 텍스트를 입력받아 요약텍스트를 출력하는 함수 생성

In [12]:
import os
import torch
from tqdm import tqdm

from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
from transformers import AutoModel, AutoConfig, AutoTokenizer

log_dict['TimeLog']['4. 요약출력 함수 생성'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print('TimeLog:', log_dict['TimeLog']['4. 요약출력 함수 생성'])

finetune_model_path = os.path.join(log_dict['파라미터']['model_path'], log_dict['파라미터']['checkpoint'])
config = AutoConfig.from_pretrained(finetune_model_path)
model = BartForConditionalGeneration.from_pretrained(finetune_model_path, config=config).to('cuda')
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

_ = model.eval()

def generate_summary(input_text: str, model, tokenizer):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    summary_ids = model.generate(
        torch.tensor(input_ids),  
        num_beams=log_dict['파라미터']['num_beams'],  
        max_length=log_dict['파라미터']['max_length'],  
        eos_token_id=log_dict['파라미터']['eos_token_id'], 
        length_penalty=log_dict['파라미터']['length_penalty']
    )
    summary = tokenizer.decode(summary_ids[0].squeeze().tolist(), skip_special_tokens=True)
    return summary

TimeLog: 2024-02-21 14:50:31


# 5. 요약평가 함수 생성 - Rouge-L

In [13]:
from rouge_score import rouge_scorer

log_dict['TimeLog']['5. 요약평가 함수 생성'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print('TimeLog:', log_dict['TimeLog']['5. 요약평가 함수 생성'])

def get_score(target, prediction):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3', 'rougeL'], use_stemmer=False)
    scores = scorer.score(target=target, prediction=prediction)
    return scores['rougeL'].precision, scores['rougeL'].recall, scores['rougeL'].fmeasure

TimeLog: 2024-02-21 14:50:34


# 6. 모델평가
- 학습된 모델을 바탕으로 test 데이터의 요약문을 에측 후 저장
- 모든 테스트 데이터에 대한 요약문이 생성되면 rouge-L 로 평가

In [14]:
import torch
from tqdm import tqdm

log_dict['TimeLog']['6. 모델평가 - 시작'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print('TimeLog: 모델평가 시작', log_dict['TimeLog']['6. 모델평가 - 시작'])

results = []
for doc in tqdm(test_dataset):
    pred = generate_summary(input_text=doc['summ_contxt'], model=model, tokenizer=tokenizer)
    
    truth = doc['summ_pass']
    truth_encoded = ' '.join([str(token) for token in tokenizer.encode(doc['summ_pass'])])
    pred_encoded = ' '.join([str(token) for token in tokenizer.encode(pred)])
    score = get_score(truth_encoded, pred_encoded)
    results.append(
        {
            'id': doc['id'],
            'truth': truth,
            'pred': pred,
            'score': {
                'precision': score[0],
                'recall': score[1],
                'fmeasure': score[2]
            }
        }
    )

result_each_text = []
for result in results:
    log_dict['문제당 개별 결과 값'].append(result)

log_dict['TimeLog']['6. 모델평가 - 종료'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print()
print()
print('TimeLog: 모델평가 종료', log_dict['TimeLog']['6. 모델평가 - 종료'])

TimeLog: 모델평가 시작 2024-02-21 14:50:34


  0%|          | 0/6651 [00:00<?, ?it/s]/tmp/ipykernel_1483/458175977.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(input_ids),
100%|██████████| 6651/6651 [1:29:32<00:00,  1.24it/s]



TimeLog: 모델평가 종료 2024-02-21 16:20:07


# 7. 모델 평가결과 계산

In [15]:
import json
from datetime import datetime


log_dict['TimeLog']['7. 모델 평가결과 계산'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print('TimeLog:', log_dict['TimeLog']['7. 모델 평가결과 계산'])

scores = [[], [], []]
for doc in results:
    score = doc['score']
    scores[0].append(score['precision'])
    scores[1].append(score['recall'])
    scores[2].append(score['fmeasure'])

average_precision = sum(scores[0]) / len(scores[0])
average_recall = sum(scores[1]) / len(scores[1])
average_fmeasure = sum(scores[2]) / len(scores[2])

log_dict['계산할 때 사용된 값'] = {
    'avg_precision': average_precision,
    'avg_recall': average_recall
}

log_dict['최종 결과 값'] = {
    '(rouge-L) avg_fmeasure': average_fmeasure
}


TimeLog: 2024-02-21 16:20:07


In [16]:
log_dict['TimeLog']['종료'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print('TimeLog: 종료', log_dict['TimeLog']['종료'])
with open('%s-모델성능평가수행로그.json' % log_dict['파라미터']['model_path'].replace('result/', '').replace('/', '').strip(), 'w') as outfile:
    json.dump(log_dict, outfile, ensure_ascii=False, indent=4)

TimeLog: 종료 2024-02-21 16:20:07
